In [1]:
import sys

# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

The sys.setrecursionlimit function is used to increase the recursion limit, which helps prevent potential recursion errors when running complex models with deep nested functions or when using certain libraries like TensorFlow.

- **`keras_tuner`**: Used for hyperparameter tuning.
- **`Sequential`**: A linear stack of layers in Keras.
- **`Dense`**, **`Flatten`**: Common Keras layers.
- **`mnist`**: The MNIST dataset, a standard dataset for image classification.
- **`Adam`**: An optimizer in Keras.

In [2]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only

In [ ]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')